# Deep Features Extraction

## Collect Data

In [4]:
modeldir='Friday26June_DenseNet169_0.911_Adam'

In [5]:
historyfilename='model/'+modeldir+'/history_model_norm.json'
modelweights='model/'+modeldir+'/model_weights_norm.h5'
BATCH_SIZE=8
thevars='model/'+modeldir+'/extractedfeatures_norm.pickle'

In [6]:
from sklearn.metrics import cohen_kappa_score, accuracy_score
import json
import math
import os
import cv2
from PIL import Image
import numpy as np
from keras import layers
from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam,SGD
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
#from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from sklearn.utils import class_weight, shuffle
from keras.losses import binary_crossentropy
from keras.applications.resnet50 import preprocess_input
import keras.backend as K
import tensorflow as tf
from sklearn.metrics import f1_score, fbeta_score
from keras.utils import Sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")






IMG_SIZE=224

In [ ]:



#The image names are found in the data-split folder
filename_train_covid='../Data-split/COVID/trainCT_COVID.txt'
filename_val_covid='../Data-split/COVID/valCT_COVID.txt'
filename_test_covid='../Data-split/COVID/testCT_COVID.txt'

filename_train_no_covid='../Data-split/NonCOVID/trainCT_NonCOVID.txt'
filename_val_no_covid='../Data-split/NonCOVID/valCT_NonCOVID.txt'
filename_test_no_covid='../Data-split/NonCOVID/testCT_NonCOVID.txt'


# In[5]:


path_covid='../Images-processed/CT_COVID/CT_COVID/'
path_non_covid='../Images-processed/CT_NonCOVID/CT_NonCOVID/'


# In[6]:



#prepare training data
with open(filename_train_covid) as f:
    covidimages_train_path = f.read().splitlines()

covidimages_train_path=[path_covid+s for s in covidimages_train_path]
covidclasses=[1]*len(covidimages_train_path)
print("Number of COVID images in train dataset is :"+ str(len(covidclasses)))

with open(filename_train_no_covid) as f:
    no_covidimages_train_path = f.read().splitlines()
non_covidimages_train_path=[path_non_covid+s for s in no_covidimages_train_path]
non_covidclasses=[0]*len(non_covidimages_train_path)
print("Number of Non COVID images in train dataset is :"+ str(len(non_covidclasses)))

trainpaths=covidimages_train_path+non_covidimages_train_path
Y_train=covidclasses+non_covidclasses

print("training samples :"+ str(len(Y_train)))


# In[7]:



#prepare val data
with open(filename_val_covid) as f:
    covidimages_val_path = f.read().splitlines()

covidimages_val_path=[path_covid+s for s in covidimages_val_path]
covidclasses=[1]*len(covidimages_val_path)
print("Number of COVID images in val dataset is :"+ str(len(covidclasses)))

with open(filename_val_no_covid) as f:
    no_covidimages_val_path = f.read().splitlines()
non_covidimages_val_path=[path_non_covid+s for s in no_covidimages_val_path]
non_covidclasses=[0]*len(non_covidimages_val_path)
print("Number of Non COVID images in val dataset is :"+ str(len(non_covidclasses)))

valpaths=covidimages_val_path+non_covidimages_val_path
Y_val=covidclasses+non_covidclasses

print("valing samples :"+ str(len(Y_val)))


# In[8]:



#prepare test data
with open(filename_test_covid) as f:
    covidimages_test_path = f.read().splitlines()

covidimages_test_path=[path_covid+s for s in covidimages_test_path]
covidclasses=[1]*len(covidimages_test_path)
print("Number of COVID images in test dataset is :"+ str(len(covidclasses)))

with open(filename_test_no_covid) as f:
    no_covidimages_test_path = f.read().splitlines()
non_covidimages_test_path=[path_non_covid+s for s in no_covidimages_test_path]
non_covidclasses=[0]*len(non_covidimages_test_path)
print("Number of Non COVID images in test dataset is :"+ str(len(non_covidclasses)))

testpaths=covidimages_test_path+non_covidimages_test_path
Y_test=covidclasses+non_covidclasses

print("testing samples :"+ str(len(Y_test)))


# In[9]:




def equalize_light(image, limit=2, grid=(16,16), gray=False):
    if (len(image.shape) == 2):
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        gray = True
    
    clahe = cv2.createCLAHE(clipLimit=limit, tileGridSize=grid)
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))

    image = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    if gray: 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    return np.uint8(image)

def normalize(x):
    """
        argument
            - x: input image data in numpy array [32, 32, 3]
        return
            - normalized x 
    """
    min_val = np.min(x) #typically will be zero
    max_val = np.max(x) #typically will be 255
    x = (x-min_val) / (max_val-min_val)
    x[x<0.8]=0
    return x
def load_ben_color(path, sigmaX=10 ):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = equalize_light(image,3,(5,5))
    #image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
    image=normalize(image)  
    return image




N = len(trainpaths)
X_train = np.empty((N, IMG_SIZE, IMG_SIZE, 3), dtype=np.float64)

for i, image_path in enumerate(tqdm(trainpaths)):
    X_train[i, :, :, :] = load_ben_color(image_path,sigmaX=10)


# In[14]:


print(X_train.shape)


# In[15]:


N_val = len(valpaths)
X_val = np.empty((N_val, IMG_SIZE, IMG_SIZE, 3), dtype=np.float64)

for i, image_path in enumerate(tqdm(valpaths)):
    X_val[i, :, :, :] = load_ben_color(image_path,sigmaX=10)
print(X_val.shape)


# In[16]:


N_test = len(testpaths)
X_test = np.empty((N_test, IMG_SIZE, IMG_SIZE, 3), dtype=np.float64)

for i, image_path in enumerate(tqdm(testpaths)):
    X_test[i, :, :, :] = load_ben_color(image_path,sigmaX=10)
print(X_test.shape)


## Import Extracted Features

In [7]:
import pickle

#with open("densenet121/extractedfeatures.pickle", "wb") as f:
#    pickle.dump((Train_Features,Train_Target,Val_Features,Val_Target,Test_Features,Test_Target,model,model_extracted), f)

with open(thevars, "rb") as f:
    Train_Features,Train_Target,Val_Features,Val_Target,Test_Features,Test_Target,model,model_extracted = pickle.load(f) 

In [24]:
Y_test=Test_Target

## DenseNet169 output

In [ ]:
preds = model.predict(X_test, verbose=2)
from sklearn.metrics import accuracy_score,f1_score,roc_curve,auc
print(accuracy_score(Y_test,np.round(preds)))
print(f1_score(Y_test,np.round(preds)))
fpr, tpr, thresholds = roc_curve(Y_test, preds)
print("AUC: " + str(auc(fpr, tpr)))

In [ ]:
history_dict={"val_loss": [0.6326061271257319, 0.5416921238272877, 0.4986336695945869, 0.49928065698783275, 0.5944412969431635, 0.5316322253606582, 0.5619842170785039, 0.4867947189674034, 0.5710916241662483, 0.5526685400597625, 0.5170543382291571, 0.4294330312286393, 0.6615542507083234, 0.5495079302749897, 0.46326644586051924, 0.4509483315682007, 0.5450441941492639, 0.45706215440848114, 0.47182994102269915], "val_acc": [0.6610169491525424, 0.7457627118644068, 0.7711864406779662, 0.7372881355932204, 0.7372881355932204, 0.7457627118644068, 0.7372881355932204, 0.7796610169491526, 0.7203389830508474, 0.7796610169491526, 0.7457627118644068, 0.8305084745762712, 0.7203389830508474, 0.7372881355932204, 0.8050847457627118, 0.8050847457627118, 0.7711864406779662, 0.8220338983050848, 0.788135593220339], "loss": [0.6525186945410336, 0.5326157165976132, 0.4295450732287239, 0.37116500588024365, 0.3361167902104995, 0.28402104167377246, 0.3148369348750395, 0.2743631196022034, 0.2774100516824161, 0.2637738080585704, 0.237877601034501, 0.2554569693172679, 0.24783022123224596, 0.20921514048295864, 0.23769445966271793, 0.2305149124650394, 0.2021147998641519, 0.2062181026795331, 0.20249913944917566], "acc": [0.611764705882353, 0.7458823529411764, 0.8329411764705882, 0.8917647058823529, 0.9152941176470588, 0.9435294117647058, 0.9035294117647059, 0.9458823529411765, 0.9435294117647058, 0.9552941176470588, 0.971764705882353, 0.9435294117647058, 0.9482352941176471, 0.9811764705882353, 0.9647058823529412, 0.9647058823529412, 0.9835294117647059, 0.9741176470588235, 0.9788235294117648]}

In [ ]:
#historyfilename ='model/MondayDesneNet169/model_weights.h5'
#history_dict = json.load(open(historyfilename, 'r'))

history_df = pd.DataFrame(history_dict)
history_df.to_csv('history.csv')
history_df[['loss', 'val_loss']].plot()
history_df[['acc', 'val_acc']].plot()

## Accuracy function

In [8]:
from sklearn.metrics import confusion_matrix
def measure_per(y_val,preds,num_classes):
    print(accuracy_score(y_val,preds))
    print(cohen_kappa_score(y_val,preds))

    
    df_cm = pd.DataFrame(cm, range(num_classes),
                              range(num_classes))
    print(df_cm)

In [9]:
num_classes=2
# Creating a function to report confusion metrics
def confusion_metrics (truvalues,predictions):
# save confusion matrix and slice into four pieces
    conf_matrix = confusion_matrix(truvalues, predictions)
    df_cm = pd.DataFrame(conf_matrix, range(num_classes),
                              range(num_classes))
    print(df_cm)
    TP = conf_matrix[1][1]
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    print('True Positives:', TP)
    print('True Negatives:', TN)
    print('False Positives:', FP)
    print('False Negatives:', FN)
    
    # calculate accuracy
    conf_accuracy = (float (TP+TN) / float(TP + TN + FP + FN))
    
    # calculate mis-classification
    conf_misclassification = 1- conf_accuracy
    
    # calculate the sensitivity
    conf_sensitivity = (TP / float(TP + FN))
    # calculate the specificity
    conf_specificity = (TN / float(TN + FP))
    
    # calculate precision
    conf_precision = (TN / float(TN + FP))
    # calculate f_1 score
    conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))
    print('-'*50)
    print(f'Accuracy: {round(conf_accuracy,2)}') 
    print(f'Mis-Classification: {round(conf_misclassification,2)}') 
    print(f'Sensitivity: {round(conf_sensitivity,2)}') 
    print(f'Specificity: {round(conf_specificity,2)}') 
    print(f'Precision: {round(conf_precision,2)}')
    print(f'f_1 Score: {round(conf_f1,2)}')

## XGBoost

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
X_train_minmax = min_max_scaler.fit_transform(Train_Features)
X_val_minmax = min_max_scaler.transform(Val_Features)
X_Test_minmax = min_max_scaler.transform(Test_Features)

In [ ]:
np.min(X_train_minmax[1])

In [20]:
from xgboost import XGBClassifier,XGBRegressor
from xgboost import plot_importance,plot_tree
eval_set = [(Val_Features, Val_Target)]

objective='binary:logistic'
eval_metric=['error']


xgb = XGBClassifier(objective=objective,learning_rate=0.12,
                   n_estimators=200,subsample=1,
                   max_depth=11,n_jobs=3)#binary:logistic

xgb.fit(Train_Features, Train_Target,eval_metric=eval_metric, eval_set=eval_set,verbose=False,early_stopping_rounds=50)

preds_xgb = xgb.predict(Test_Features)

from sklearn.metrics import accuracy_score,f1_score,roc_curve,auc
fpr, tpr, thresholds = roc_curve(Test_Target, xgb.predict_proba(Test_Features)[:,1])
print("AUC: " + str(auc(fpr, tpr)))

AUC: 0.8763848396501457


In [21]:
confusion_metrics(Test_Target,preds_xgb)

    0   1
0  86  19
1  21  77
True Positives: 77
True Negatives: 86
False Positives: 19
False Negatives: 21
--------------------------------------------------
Accuracy: 0.8
Mis-Classification: 0.2
Sensitivity: 0.79
Specificity: 0.82
Precision: 0.82
f_1 Score: 0.8


In [ ]:
#with open("bestXGBoostmodel.pickle", "wb") as f:
#    pickle.dump((xgb), f)

## RF

In [ ]:
with open("bestRFmodel/bestRFmodel.pickle", "rb") as f:
    rfa = pickle.load(f) 

In [27]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=10, n_estimators=400)
rf.fit(Train_Features, Train_Target)
preds_rf = rf.predict(Test_Features)
fpr, tpr, thresholds = roc_curve(Y_test, rf.predict_proba(Test_Features)[:,1])
print("AUC: " + str(auc(fpr, tpr)))

AUC: 0.8961613216715256


In [28]:
confusion_metrics(Y_test,preds_rf)

    0   1
0  85  20
1  19  79
True Positives: 79
True Negatives: 85
False Positives: 20
False Negatives: 19
--------------------------------------------------
Accuracy: 0.81
Mis-Classification: 0.19
Sensitivity: 0.81
Specificity: 0.81
Precision: 0.81
f_1 Score: 0.81


In [ ]:
imp_rf = rf.feature_importances_

## Bagging Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)
bdt = BaggingClassifier(n_estimators=200,base_estimator=clf).fit(Train_Features, Train_Target)
#Predict the response for test dataset
preds_bdt = bdt.predict(Test_Features)

fpr, tpr, thresholds = roc_curve(Y_test, bdt.predict_proba(Test_Features)[:,1])
print("AUC: " + str(auc(fpr, tpr)))

In [ ]:
confusion_metrics(Y_test,preds_bdt)

In [ ]:
with open("bestbdtmodel.pickle", "wb") as f:
    pickle.dump((bdt), f)

## Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=200)
clf.fit(Train_Features,Train_Target)
#Predict the response for test dataset
preds_adaboost = clf.predict(Test_Features)

fpr, tpr, thresholds = roc_curve(Y_test, clf.predict_proba(Test_Features)[:,1])
print("AUC: " + str(auc(fpr, tpr)))

In [ ]:
confusion_metrics(Y_test,preds_adaboost)

In [ ]:
with open("bestadaboostmodel.pickle", "wb") as f:
    pickle.dump((clf), f)

## GBDT

In [ ]:
import lightgbm
train_data = lightgbm.Dataset(Train_Features, label=np.squeeze(Train_Target))
val_data = lightgbm.Dataset(Val_Features, label=np.squeeze(Val_Target))

parameters = {
    'objective': 'cross_entropy',
    'boosting': 'gbdt',#'gbdt' 'dart'
    'num_leaves': 21,
    'feature_fraction': 0.99,
    'bagging_fraction': 0.99,
    'bagging_freq': 30,  'learning_rate': 0.1,
    'verbose': 2,
}

gbm = lightgbm.train(parameters,
                       train_data,
                       valid_sets=val_data,
                       num_boost_round=1000,
                       early_stopping_rounds=100)



In [ ]:
preds_gbdt=gbm.predict(Test_Features)
fpr, tpr, thresholds = roc_curve(Y_test, preds_gbdt)
print("AUC: " + str(auc(fpr, tpr)))

In [ ]:
confusion_metrics(Y_test,np.round(preds_gbdt))

In [ ]:
with open("bestlightgbmmodel.pickle", "wb") as f:
    pickle.dump((gbm), f)

## Dart

In [ ]:
import lightgbm
train_data = lightgbm.Dataset(Train_Features, label=np.squeeze(Train_Target))
val_data = lightgbm.Dataset(Val_Features, label=np.squeeze(Val_Target))

parameters = {
    'objective': 'binary',
    'boosting': 'dart',#'gbdt' 'dart'
    'num_leaves': 31,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'bagging_freq': 30,
    'verbose': 2,
}

dart = lightgbm.train(parameters,
                       train_data,
                       valid_sets=val_data,
                       num_boost_round=1000,
                       early_stopping_rounds=100)



In [ ]:
preds_dart=dart.predict(Test_Features)
fpr, tpr, thresholds = roc_curve(Y_test, preds_dart)
print("AUC: " + str(auc(fpr, tpr)))

In [ ]:
confusion_metrics(Y_test,np.round(preds_dart))

In [ ]:
with open("bestdartmmodel.pickle", "wb") as f:
    pickle.dump((dart), f)

# Cluster

In [ ]:
# import KMeans
from sklearn.cluster import KMeans

In [ ]:
# create kmeans object
kmeans = KMeans(n_clusters=2)
# fit kmeans object to data
kmeans.fit(Train_Features)
# print location of clusters learned by kmeans object
print(kmeans.cluster_centers_)

In [ ]:
y_km = kmeans.fit_predict(Train_Features)

In [ ]:
y_km

In [ ]:
s = kmeans.fit_predict(Test_Features)

In [ ]:
print(Y_test)

In [ ]:
print(s)

In [ ]:
confusion_metrics(Y_test,s)

# Check Features

In [ ]:
import pandas as pd

In [ ]:
di={'actual':Y_test,'preds_xgboost':preds_xgb,'preds_rf':preds_rf,
   'preds_bdt':preds_bdt,'preds_adaboost':preds_adaboost,'preds_gbdt':np.round(preds_gbdt),'preds_dart':np.round(preds_dart)}

In [ ]:
df=pd.DataFrame(di)

In [ ]:
df

# SVM

In [ ]:
from sklearn.svm import SVC,LinearSVC, NuSVC
svm = SVC(kernel='rbf',C=1)
svm.fit(Train_Features, Train_Target)#sometimes labels instead
print('fitting done !!!')


preds=svm.predict(Test_Features)
confusion_metrics(Y_test,preds)
fpr, tpr, thresholds = roc_curve(Y_test, preds)
print("AUC: " + str(auc(fpr, tpr)))

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=300)
x_train_pca = pca.fit_transform(Train_Features)
x_test_pca=pca.transform(Test_Features)

In [ ]:
from sklearn import model_selection
from sklearn.svm import NuSVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import neighbors

kfold = model_selection.KFold(n_splits=5, random_state=5)
# create the sub models
estimators = []
model1 = SVC(kernel='rbf')
estimators.append(('model1', model1))

model2 = neighbors.KNeighborsClassifier(13, weights='distance')
estimators.append(('model2', model2))


model3 = GaussianNB()
estimators.append(('model3', model3))
# create the ensemble model
ensemble = VotingClassifier(estimators)

ensemble.fit(Train_Features, Train_Target)
#results = model_selection.cross_val_score(ensemble, x_train, y_train, cv=kfold)
#print(results.mean())
preds=ensemble.predict(Test_Features)
confusion_metrics(Y_test,preds)

In [ ]:
e=ensemble.predict_proba(Test_Features)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_test, e)
print("AUC: " + str(auc(fpr, tpr)))

In [ ]:
from sklearn.ensemble import BaggingClassifier
bc = BaggingClassifier(base_estimator=SVC(kernel='rbf'),n_estimators=3, random_state=0).fit(Train_Features, Train_Target)
preds=bc.predict(Test_Features)
confusion_metrics(Y_test,preds)

In [ ]:
e=bc.predict_proba(Test_Features)[:,1]
fpr, tpr, thresholds = roc_curve(Y_test, e)
print("AUC: " + str(auc(fpr, tpr)))

In [ ]:
basec=SVC(probability=True, kernel='rbf')
# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=5,
                         learning_rate=0.1,base_estimator=basec).fit(Train_Features, Train_Target)
preds=abc.predict(Test_Features)
confusion_metrics(Y_test,preds)